In [1]:
import os
import time

import samlab.dashboard

from selenium.webdriver import ActionChains, Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support.expected_conditions import presence_of_element_located as element_located

dashboard = samlab.dashboard.Server(config=False, quiet=True)
dashboard.ready(timeout=10)
print(dashboard.uri)

options = Options()
options.headless = True
options.add_argument("--width=1024")
options.add_argument("--height=768")

browser = Firefox(options=options)
browser.get(dashboard.uri)
Wait(browser, timeout=10).until(element_located((By.ID, "samlab-dashboard-favorites-menu")))

os.makedirs("dashboard", exist_ok=True)


def screenshot(path, delay=0.5):
    path = os.path.join(os.getcwd(), "dashboard", path)
    print(f"Saving screenshot {path}")
    time.sleep(delay)
    browser.save_screenshot(os.path.join(os.getcwd(), path))
    time.sleep(delay)


favorites_menu = browser.find_element_by_css_selector("#samlab-dashboard-favorites-menu")
datasets_menu = browser.find_element_by_css_selector("#samlab-dashboard-datasets-menu")
operations_menu = browser.find_element_by_css_selector("#samlab-dashboard-operations-menu")

# # Default home page.
# screenshot("welcome.png")

# # Show the operations menu.
# operations_menu.find_element_by_css_selector("button").click()
# screenshot("operations-menu.png")
# operations_menu.find_element_by_css_selector("button").click()

# # Show the datasets menu.
# datasets_menu.find_element_by_css_selector("button").click()
# screenshot("empty-datasets-menu.png")
# datasets_menu.find_element_by_css_selector("button").click()

# # Show the favorites menu.
# favorites_menu.find_element_by_css_selector("button").click()
# screenshot("empty-favorites-menu.png")
# favorites_menu.find_element_by_css_selector("button").click()

# Display the About menu item.
operations_menu.find_element_by_css_selector("button").click()
item = browser.find_element_by_link_text("About")
ActionChains(browser).move_to_element(item).perform()
time.sleep(0.5)
screenshot("before-about.png")

# Open an About widget.
item.click()
screenshot("after-about.png")

# Hover over the About widget drag region.

# Drag the About widget to a new location.

http://127.0.0.1:55397
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/before-about.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/after-about.png


In [16]:
browser.find_element_by_xpath("//h1[text()='About Samlab']/ancestor::*[contains(@class, 'grid-stack-item')]")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7c0f9c79-28cf-8a41-a533-2baaace0c2d4", element="d161de5b-0e31-6244-a247-a3d58663fe9a")>

In [ ]:
browser.quit()
dashboard.stop()

For this tutorial, let's begin by starting the server from the command line:

`$ samlab-dashboard`

The server will start, and automatically open the dashboard interface in a web browser:

<img class="browser" src="dashboard/welcome.png">

Let's examine the menus from right-to-left.  The Operations menu provides a set of generic commands for managing the dashboard layout, information lists, and displaying acknowlegements:

<img class="browser" src="dashboard/operations-menu.png">

The Datasets menu lists all of the data sources that have been registered with the server. Since we haven't registered any yet, the menu is empty: 

<img class="browser" src="dashboard/empty-datasets-menu.png">

The Favorites menu lists dashboard layouts that have been favorited for later recall or sharing with collaborators.  Once again, the list is empty, since we haven't favorited anything yet:

<img class="browser" src="dashboard/empty-favorites-menu.png">

Let's get some familiarity with the Samlab user interface.  Start by clicking on the *Operations > About* menu:

<img class="browser" src="dashboard/before-about.png">

This will open the *About Widget*:

<img class="browser" src="dashboard/after-about.png">

A Samlab *widget* is a window that you can move and resize to arrange the dashboard however you like.  To move a widget, click the mouse in the *move region* near the top of the widget, and drag the widget to a new location:
